In [1]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance as distance
from sklearn import preprocessing
from sklearn.svm import SVC

In [3]:
img_dir_list = ['PIE/PIE/1/*.jpg', 'PIE/PIE/2/*.jpg', 'PIE/PIE/3/*.jpg', \
               'PIE/PIE/4/*.jpg', 'PIE/PIE/5/*.jpg', 'PIE/PIE/6/*.jpg', \
               'PIE/PIE/7/*.jpg', 'PIE/PIE/8/*.jpg', 'PIE/PIE/9/*.jpg', \
               'PIE/PIE/10/*.jpg', 'PIE/PIE/11/*.jpg', 'PIE/PIE/12/*.jpg', \
               'PIE/PIE/13/*.jpg', 'PIE/PIE/14/*.jpg', 'PIE/PIE/15/*.jpg', \
               'PIE/PIE/16/*.jpg', 'PIE/PIE/17/*.jpg', 'PIE/PIE/18/*.jpg', \
               'PIE/PIE/19/*.jpg', 'PIE/PIE/20/*.jpg']
image_list = []
image_own = []
train_list = []
train_own = []
test_list = []
test_own = []
image_label = []
own_label = [20] * 10

for i in range (0, 20):
    for filename in glob.glob(img_dir_list[i]):
        im = Image.open(filename)
        arr = np.array(im).flatten()
        image_list.append(arr)
        image_label.append(i)
for filename in glob.glob('own_image/*.jpg'):
    im = Image.open(filename)
    arr = np.array(im).flatten()
    image_own.append(arr)
    
image_list = np.asarray(image_list)
image_own = np.asarray(image_own)
image_label = np.asarray(image_label)
own_label = np.asarray(own_label)

train_list, test_list, train_label, test_label = train_test_split(image_list, image_label, test_size = 0.3)
train_list = np.asarray(train_list)
test_list = np.asarray(test_list)
train_label = np.asarray(train_label)
test_label = np.asarray(test_label)

train_own, test_own, train_own_label, test_own_label = train_test_split(image_own, own_label, test_size = 0.3)
train_own = np.asarray(train_own)
test_own = np.asarray(test_own)
train_own_label = np.asarray(train_own_label)
test_own_label = np.asarray(test_own_label)

random_indices = np.arange(0, train_list.shape[0])
np.random.shuffle(random_indices)
train_sample = train_list[random_indices[:500]]
train_sample_label = train_label[random_indices[:500]]
# print(train_sample)
train_sample = np.concatenate((train_sample, train_own))
train_sample_label = np.concatenate((train_sample_label, train_own_label))
test_list = np.concatenate((test_list, test_own))
test_label = np.concatenate((test_label, test_own_label))
# print(train_sample)
# print(train_own)

In [5]:
train_image_mean = (np.mean(train_sample, axis = 0)).astype('float')

In [7]:
def pca (image_list):
    image_mean = train_image_mean
    image_diff = image_list - image_mean
    image_diff = np.asarray(image_diff)
    covariance = np.cov(image_diff.T)
#     covariance = np.cov(image_list.T)
#         print(covariance.shape)
    eigenvalue, eigenvector = np.linalg.eig(covariance)
    idx = eigenvalue.argsort()[::-1]
    eigenvalue = np.real(eigenvalue[idx])
    eigenvector = np.real(eigenvector[:, idx])
    eigenvalue = np.asarray(eigenvalue)
    eigenvector = np.asarray(eigenvector)
#     eigenval_list.append(np.real(np.asarray(eigenvalue)))
#     eigenvec_list.append(np.real(np.asarray(eigenvector)))
#     eigenval_list = np.asarray(eigenval_list)
#     eigenvec_list = np.asarray(eigenvec_list)
    return eigenvalue, eigenvector

In [8]:
def get_eigen_pair(eigenvalue, eigenvector):
    eigen_pairs = [(eigenvalue[i], eigenvector[:,i]) for i in range(len(eigenvalue))]
    eigen_pairs.sort(key=lambda k: k[0], reverse = True)
    return eigen_pairs

def reduce_dim(image_list, eigenvalue, eigenvector, dim):
    eigenpair = get_eigen_pair(eigenvalue, eigenvector)
    w = np.hstack((eigenpair[0][1][:, np.newaxis], eigenpair[1][1][:, np.newaxis]))
    if (dim > 2):
        for i in range (2, dim):
            w = np.hstack((w, eigenpair[i][1][:, np.newaxis]))
    reduce_dim_result = image_list.dot(w)
    return reduce_dim_result.T

In [9]:
eigenvalue_test, eigenvector_test = pca(test_list)
eigenvalue_train, eigenvector_train = pca(train_sample)